In [1]:
import pandas as pd
from os import path, environ, makedirs
import requests as re
from io import BytesIO
from dotenv import load_dotenv
from unidecode import unidecode
import numpy as np

In [2]:
load_dotenv()

True

# Carregando os dados extraídos no notebook anterior

Neste notebook, vamos utilizar os dados extraídos e salvos pelo notebook `05 ObservaSampa - extração.ipynb`.

Primeiro, vamos carregar os dados de subprefeituras que serão utilizados no Qlik Sense.

In [3]:
output_dir = path.join('dados', 'urbanismo')
if not path.exists(output_dir):
    makedirs(output_dir)

In [4]:
dist_mun = pd.read_csv(path.join(output_dir, 'Distrito.csv'),
                        sep=';', decimal=',', encoding='utf8', dtype=str)

In [5]:
dist_censo = pd.read_csv(path.join(output_dir, 'Setor-censitario.csv'),
                          sep=';', decimal=',', encoding='utf8', dtype=str)

In [6]:
subs = pd.read_csv(path.join(output_dir, 'Subprefeituras.csv'),
                              sep=';', decimal=',', encoding='utf8', dtype=str)

## Tratamento do Indicador domicilios não ocupados (censo 2022)

In [7]:
def construir_df_ido(dist_mun, dist_censo, subs):
    dist_mun = dist_mun.loc[:, ~dist_mun.columns.duplicated(keep='last')].copy()
    dist_censo = dist_censo[dist_censo["CD_MUN"].astype(str) == "3550308"].copy()
    dist_censo["nm_distrito_municipal"] = dist_censo["NM_DIST"].apply(unidecode).str.upper()
    dist_mun = dist_mun.merge(
        dist_censo[["nm_distrito_municipal", "v0003", "v0007"]],
        on="nm_distrito_municipal",
        how="left"
    ).rename(columns={"v0003": "total_domicilios_particulares", "v0007": "domicilios_particulares_ocupados"})
    for c in ["total_domicilios_particulares", "domicilios_particulares_ocupados"]:
        dist_mun[c] = pd.to_numeric(dist_mun[c], errors="coerce")
    dist_mun["domicilios_particulares_nao_ocupados"] = dist_mun["total_domicilios_particulares"] - dist_mun["domicilios_particulares_ocupados"]
    dist_mun["cd_subprefeitura"] = pd.to_numeric(dist_mun["cd_identificador_subprefeitura"], errors="coerce").astype("Int64").astype(str).str.zfill(2)
    subs = subs.copy()
    subs["cd_subprefeitura"] = pd.to_numeric(subs["cd_subprefeitura"], errors="coerce").astype("Int64").astype(str).str.zfill(2)
    dom_subs = dist_mun.groupby("cd_subprefeitura", as_index=False).agg({
        "total_domicilios_particulares": "sum",
        "domicilios_particulares_ocupados": "sum",
        "domicilios_particulares_nao_ocupados": "sum"
    })
    df_ido = subs.merge(dom_subs, on="cd_subprefeitura", how="left")[[
        "cd_subprefeitura",
        "nm_subprefeitura",
        "domicilios_particulares_nao_ocupados",
        "domicilios_particulares_ocupados",
        "total_domicilios_particulares"
    ]].copy()
    df_ido["nm_subprefeitura"] = df_ido["nm_subprefeitura"].astype(str).str.strip()
    df_ido["tx_nao_ocupados_pct"] = np.where(
        pd.to_numeric(df_ido["total_domicilios_particulares"], errors="coerce").gt(0),
        df_ido["domicilios_particulares_nao_ocupados"] / df_ido["total_domicilios_particulares"] * 100,
        np.nan
    )
    return df_ido.sort_values("cd_subprefeitura").reset_index(drop=True)

In [8]:
df_ido = construir_df_ido(dist_mun, dist_censo, subs)

In [9]:
_dest = path.join("data_output", "urbanismo")
if not path.exists(_dest):
    makedirs(_dest)

In [10]:
df_ido.to_csv(
    path.join(_dest, "Indicador domicilios não ocupados (censo 2022).csv"),
    index=False,
    sep=";",
    decimal=",",
    encoding="latin1"
)

In [11]:
df_ido

,cd_subprefeitura,nm_subprefeitura,domicilios_particulares_nao_ocupados,domicilios_particulares_ocupados,total_domicilios_particulares,tx_nao_ocupados_pct
0,01,PERUS,7826,56874,64700,12.095827
1,02,PIRITUBA-JARAGUA,25143,175623,200766,12.523535
2,03,FREGUESIA-BRASILANDIA,19562,138060,157622,12.410704
3,04,CASA VERDE-CACHOEIRINHA,16151,113822,129973,12.426427
4,05,SANTANA-TUCURUVI,22676,125151,147827,15.339552
5,06,JACANA-TREMEMBE,13860,99015,112875,12.279070
6,07,VILA MARIA-VILA GUILHERME,15956,104177,120133,13.281946
7,08,LAPA,21194,141589,162783,13.019787
8,09,SE,54320,202905,257225,21.117699
9,10,BUTANTA,25508,178454,203962,12.506251


# Armazenamento

Finalmente, vamos exportar os dados em formato csv compatível com o Qlik e no padrão do excel para português do Brasil.

In [12]:
base_path = path.join('data_output', 'urbanismo')

if not path.exists(base_path):
    makedirs(base_path)

for name, df in [
                 ('indicador_domicilios_nao_ocupados', df_ido),
                 ]:
    filepath = path.join(base_path, f'{name}.csv')

    df.to_csv(filepath,
              index=False,
              sep=';',
              decimal=',',
              encoding='latin1')